# <font color = "red">Classificação de Risco de Crédito</font>

## Uma análise Comparativa Para Prever Inadimplência 

<font color = "red">Problema de Negócio:</font> A partir de dados históricos, irei verificar os padrões que levaram os clientes de uma instituição financeira, a adentrarem, ou não, na situação de inadimplência, para posteriormente criar um 


<font color = "red">Meta de Acurácia Final:</font> ACC mínimo de 88%

In [1]:
# Imports 
import sys
import os
import pandas as pd
import numpy as np


sys.path.append(os.path.abspath('..'))
from utils import *

In [2]:
df_original = pd.read_csv("C:/Projetos Pessoais/DataScience/analise_Risco_Credito/data/risco_credito_original.csv")

In [3]:
df_original.columns

Index(['person_age', 'person_income', 'person_home_ownership',
       'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt',
       'loan_int_rate', 'loan_status', 'loan_percent_income',
       'cb_person_default_on_file', 'cb_person_cred_hist_length'],
      dtype='object')

In [4]:
df_original.shape

(32581, 12)

In [5]:
df_original.sample(10)

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
21929,27,44400,MORTGAGE,10.0,VENTURE,C,4000,13.99,1,0.09,Y,7
28358,29,200000,RENT,14.0,PERSONAL,B,16000,12.69,0,0.08,N,7
16517,22,45000,RENT,0.0,EDUCATION,C,4000,NaN,0,0.09,Y,3
18129,35,15000,OWN,3.0,HOMEIMPROVEMENT,A,3700,9.32,1,0.25,N,6
30910,48,40000,RENT,3.0,MEDICAL,A,10000,NaN,0,0.25,N,17
16490,24,70000,OWN,NaN,EDUCATION,A,6000,5.42,0,0.09,N,2
20547,32,41000,MORTGAGE,2.0,PERSONAL,D,8000,15.05,0,0.20,N,9
30462,38,24000,RENT,5.0,MEDICAL,B,7200,10.65,0,0.30,N,14
26784,32,40277,RENT,5.0,MEDICAL,B,15000,11.71,1,0.37,N,8
13263,23,90000,MORTGAGE,1.0,EDUCATION,A,4000,5.42,0,0.04,N,3


#### <font color = "red">Verificando valores únicos, para realizar mapeamento das variaveis

In [6]:
df_original.person_home_ownership.unique()

array(['RENT', 'OWN', 'MORTGAGE', 'OTHER'], dtype=object)

In [7]:
df_original.loan_intent.unique()

array(['PERSONAL', 'EDUCATION', 'MEDICAL', 'VENTURE', 'HOMEIMPROVEMENT',
       'DEBTCONSOLIDATION'], dtype=object)

#### <font color = "red">Realização de mapeamento no conjunto de dados

In [8]:
# Mudando nome das variáveis
df_mapeado = df_original.rename(columns=map_titulo_variaveis)
df_mapeado.head()

,Idade,Renda_Anual,Tipo_Moradia,Anos_no_Emprego,Motivo_Emprestimo,Grau_Emprestimo,Valor,Taxa_Juros,Status_Pagamento,Comprometimento_Renda,Inadimplencia_Historica,Duracao_Historico_Credito
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4


In [9]:
# Traduzindo valor de 2 variaveis categóricas
df_mapeado['Tipo_Moradia'] = df_mapeado['Tipo_Moradia'].map(map_tipo_moradia)
df_mapeado['Motivo_Emprestimo'] = df_mapeado['Motivo_Emprestimo'].map(map_motivo_emprestimo)


df_mapeado['Inadimplencia_Historica'] = df_mapeado['Inadimplencia_Historica'].map(map_inadimplencia_historica)

In [10]:
df_mapeado.sample(5)

,Idade,Renda_Anual,Tipo_Moradia,Anos_no_Emprego,Motivo_Emprestimo,Grau_Emprestimo,Valor,Taxa_Juros,Status_Pagamento,Comprometimento_Renda,Inadimplencia_Historica,Duracao_Historico_Credito
2302,21,31200,Hipoteca,2.0,Educação,D,1000,14.59,0,0.03,Não,2
3714,26,20000,Aluguel,0.0,Pessoal,C,4000,13.79,0,0.20,Sim,4
21666,29,50000,Próprio,3.0,Reformas Domésticas,B,4200,9.91,0,0.08,Não,5
8609,23,59004,Próprio,3.0,Empreendimento,A,4000,6.99,0,0.07,Não,3
666,25,60000,Aluguel,9.0,Educação,B,20000,NaN,1,0.33,Não,3


-----
# Exploração Inicial

In [11]:
df_mapeado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32581 entries, 0 to 32580
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Idade                      32581 non-null  int64  
 1   Renda_Anual                32581 non-null  int64  
 2   Tipo_Moradia               32581 non-null  object 
 3   Anos_no_Emprego            31686 non-null  float64
 4   Motivo_Emprestimo          32581 non-null  object 
 5   Grau_Emprestimo            32581 non-null  object 
 6   Valor                      32581 non-null  int64  
 7   Taxa_Juros                 29465 non-null  float64
 8   Status_Pagamento           32581 non-null  int64  
 9   Comprometimento_Renda      32581 non-null  float64
 10  Inadimplencia_Historica    32581 non-null  object 
 11  Duracao_Historico_Credito  32581 non-null  int64  
dtypes: float64(3), int64(5), object(4)
memory usage: 3.0+ MB


#### <font color = 'red'> Alteração dos tipos de variáveis</font>

<font color = 'red'>Status_Pagamento -> int para object (Classe)

Todos os valores int serão convertidos para float</font>

In [12]:
df_mapeado['Status_Pagamento'] = df_mapeado['Status_Pagamento'].astype("object")

In [13]:
# transformando int em float
# Nome de todas as colunas numéricas
variaveis_numericas = ['Idade','Renda_Anual','Anos_no_Emprego',
                       'Valor','Taxa_Juros','Comprometimento_Renda',
                       'Duracao_Historico_Credito']

for variaveis in variaveis_numericas:
 df_mapeado[variaveis] = df_mapeado[variaveis].astype("float")

In [14]:
df_mapeado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32581 entries, 0 to 32580
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Idade                      32581 non-null  float64
 1   Renda_Anual                32581 non-null  float64
 2   Tipo_Moradia               32581 non-null  object 
 3   Anos_no_Emprego            31686 non-null  float64
 4   Motivo_Emprestimo          32581 non-null  object 
 5   Grau_Emprestimo            32581 non-null  object 
 6   Valor                      32581 non-null  float64
 7   Taxa_Juros                 29465 non-null  float64
 8   Status_Pagamento           32581 non-null  object 
 9   Comprometimento_Renda      32581 non-null  float64
 10  Inadimplencia_Historica    32581 non-null  object 
 11  Duracao_Historico_Credito  32581 non-null  float64
dtypes: float64(7), object(5)
memory usage: 3.0+ MB


In [15]:
df_mapeado.sample(5)

,Idade,Renda_Anual,Tipo_Moradia,Anos_no_Emprego,Motivo_Emprestimo,Grau_Emprestimo,Valor,Taxa_Juros,Status_Pagamento,Comprometimento_Renda,Inadimplencia_Historica,Duracao_Historico_Credito
629,22.0,47000.0,Aluguel,5.0,Médico,D,20000.0,16.77,1,0.43,Não,3.0
14964,26.0,120000.0,Hipoteca,5.0,Educação,B,13000.0,10.99,0,0.11,Não,2.0
14873,23.0,90100.0,Hipoteca,7.0,Médico,E,21600.0,NaN,1,0.20,Sim,2.0
10256,22.0,68004.0,Aluguel,4.0,Consolidação Dívidas,A,9600.0,7.51,0,0.14,Não,2.0
8706,26.0,60000.0,Hipoteca,3.0,Educação,C,30000.0,14.27,0,0.50,Não,4.0


<font color = 'red'>Agora o conjunto de dados está pronto para a etapa de Análise Exploratória, Engenharia de Atributos e geração de Insights, que farei no notebook 02.

In [151]:
# Salvando DF 
df_mapeado.to_csv('../data/dados_mapeados2.csv', sep =',', index = False, encoding = 'utf-8')